# Using Spark to do ELT from and to S3 Data Lake

## Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek, monotonically_increasing_id
from pyspark.sql.types import *
import datetime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1658566576137_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Specify the source data path

In [2]:
song_data_path = "s3://udacity-dend/song_data/*/*/*/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
log_data_path = "s3://udacity-dend/log_data/*/*/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Specify the target data path

In [4]:
olap_tables_path = ""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create Spark Session

In [5]:
spark = SparkSession.builder.appName("Sparkify Data Lake").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process Song Data

In [6]:
song_data_df = spark.read.json(song_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
song_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [8]:
song_data_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|233.22077|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|287.92118|        1|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|1983|
|ARQ846I1187B9A7083|           null|                    |            null|Yvonne S. Moriart...|196.04853|        1|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|   0|
|AR4T2IF1187B9ADBB7|       63.9602

### Create songs_table

In [9]:
songs_table = song_data_df.dropDuplicates(['song_id']).select("song_id", "title", "artist_id", "duration", "year")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+------------------+---------+----+
|           song_id|              title|         artist_id| duration|year|
+------------------+-------------------+------------------+---------+----+
|SOAACFC12A8C140567|Supernatural Pt. II|ARNHTE41187B99289A|343.09179|   0|
|SOABVPU12AB018AA22|         Conquer Me|ARZZRK91187B9A5CA5|180.53179|2009|
|SOABWGO12AB0182725|I'm Just A Prisoner|ARCCX2V1187B99DEE6|192.88771|1976|
|SOACEFM12A8C137F81|    Somos La Ciudad|ARXQBVT11F4C83B62C|283.81995|   0|
|SOAEGIH12A6D4F8B51|     I Can't Decide|ARY2Z6Y1187B9BA126|166.53016|2006|
+------------------+-------------------+------------------+---------+----+
only showing top 5 rows

In [11]:
songs_table.write.partitionBy("year","artist_id").mode("overwrite").parquet(f"{olap_tables_path}/songs_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create artists_table

In [12]:
artists_table = song_data_df.dropDuplicates(['artist_id']).select("artist_id", col("artist_name").alias("name"),col("artist_location").alias("location"), col("artist_latitude").alias("latitude"), col("artist_longitude").alias("longitude"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------+--------+---------+
|         artist_id|                name|location|latitude|longitude|
+------------------+--------------------+--------+--------+---------+
|AR04BF811A348F050D|  Teenagers In Tokyo|        |    null|     null|
|AR04KY61187FB44E3A|         Fidel Nadal|        |    null|     null|
|AR04NH21187FB4485B|      Demise Of Eros|        |    null|     null|
|AR05KAM1187B9B6F55|           Cal Smith|Gans, OK|35.38716|-94.69444|
|AR05TTS11C8A422FEC|Westbam And The L...|        |    null|     null|
+------------------+--------------------+--------+--------+---------+
only showing top 5 rows

In [14]:
artists_table.write.mode("overwrite").parquet(f"{olap_tables_path}/artists_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process Log Data

In [15]:
log_data_df = spark.read.json(log_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
log_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [17]:
log_data_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

### Filter log_data only for page="NextSong"

In [18]:
log_data_df=log_data_df.filter(log_data_df.page=="NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create users_table

In [19]:
users_table = log_data_df.dropDuplicates(["userId"]).select(col("userId").alias("user_id"), col("firstName").alias("first_name"), col("lastName").alias("last_name"), "gender", "level")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     18|     Jacob|   Rogers|     M| free|
|     27|    Carlos|   Carter|     M| free|
|     61|    Samuel| Gonzalez|     M| free|
|      7|    Adelyn|   Jordan|     F| free|
|     26|      Ryan|    Smith|     M| free|
+-------+----------+---------+------+-----+
only showing top 5 rows

In [21]:
users_table.write.mode("overwrite").parquet(f"{olap_tables_path}/users_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create time_table

In [22]:
# Create function to convert ts (in miliseconds) to timestamp type
get_timestamp = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0),TimestampType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
log_data_df = log_data_df.withColumn("timestamp", get_timestamp("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
log_data_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|           timestamp|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:...|
|The Prodigy|Logged In|     Ryan|     M|            1|  

In [25]:
log_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

In [26]:
time_table = log_data_df.dropDuplicates(["ts"]).select(col("timestamp").alias("start_time"), hour("timestamp").alias("hour"), dayofmonth("timestamp").alias("day"),weekofyear("timestamp").alias("week"), month("timestamp").alias("month"), year("timestamp").alias("year"), dayofweek("timestamp").alias("weekday"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 15:37:...|  15| 15|  46|   11|2018|      5|
|2018-11-21 05:40:...|   5| 21|  47|   11|2018|      4|
|2018-11-28 09:35:...|   9| 28|  48|   11|2018|      4|
|2018-11-15 10:24:...|  10| 15|  46|   11|2018|      5|
|2018-11-15 22:27:...|  22| 15|  46|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [28]:
time_table.write.partitionBy("year", "month").mode("overwrite").parquet(f"{olap_tables_path}/time_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create songplays_table

In [29]:
songplays_table = log_data_df.join(song_data_df, (log_data_df.song == song_data_df.title) & (log_data_df.artist == song_data_df.artist_name)).select(monotonically_increasing_id().alias('songplay_id'), col('timestamp').alias('start_time'), year('timestamp').alias('year'),month('timestamp').alias('month'),col('userId').alias('user_id'), 'level', 'song_id','artist_id', col('sessionId').alias('session_id'),'location', col('userAgent').alias('user_agent'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+----+-----+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|songplay_id|          start_time|year|month|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-----------+--------------------+----+-----+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|          0|2018-11-15 07:31:...|2018|   11|     49| paid|SOCUITT12AB0187A32|ARKS2FE1187B99325D|       606|San Francisco-Oak...|Mozilla/5.0 (Wind...|
|          1|2018-11-15 10:44:...|2018|   11|     80| paid|SOSDZFY12A8C143718|AR748W61187B9B6AB8|       611|Portland-South Po...|"Mozilla/5.0 (Mac...|
|          2|2018-11-15 10:48:...|2018|   11|     80| paid|SOTNWCI12AAF3B2028|ARS54I31187FB46721|       611|Portland-South Po...|"Mozilla/5.0 (Mac...|
|          3|2018-11-15 10:52:...|2018|   11|     80| paid|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D

In [31]:
songplays_table.write.partitionBy("year", "month").mode("overwrite").parquet(f"{olap_tables_path}/songplays_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Analytics query example

In [32]:
songplays_table.createOrReplaceTempView("songplays_table")
artists_table.createOrReplaceTempView("artists_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
result=spark.sql('''
SELECT artists_table.artist_id as id,
    artists_table.name as artist_name,
    count(artists_table.artist_id) as count
FROM artists_table JOIN songplays_table
    ON artists_table.artist_id=songplays_table.artist_id
GROUP BY id, artist_name
ORDER BY count desc
LIMIT 10
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-----+
|                id|         artist_name|count|
+------------------+--------------------+-----+
|AR5E44Z1187B9A1D74|       Dwight Yoakam|   37|
|ARD46C811C8A414F3F|Kid Cudi / Kanye ...|   10|
|AR37SX11187FB3E164|          Ron Carter|    9|
|AR5EYTL1187B98EDA0|       Lonnie Gordon|    9|
|ARKQQZA12086C116FC|               B.o.B|    8|
|ARR3ONV1187B9A2F59|                Muse|    6|
|ARPDVPJ1187B9ADBE9|Usher featuring J...|    6|
|ARM0P6Z1187FB4D466|      Arctic Monkeys|    5|
|ARPN0Y61187B9ABAA0|          The Smiths|    4|
|ARVLXWP1187FB5B94A|     matchbox twenty|    4|
+------------------+--------------------+-----+